# Les forêts aléatoires avec python

Ce notebook est le code lié à l'article du blog de stat4decision se trouvant ici : 

https://www.stat4decision.com/fr/foret-aleatoire-avec-python/


In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

On récupère les données, il s'agit de données de vins, on essaye de prédire la couleur du vin à partir des caractérisiques physico-chimiques 

In [2]:
data = pd.read_csv("https://www.stat4decision.com/wine_data.csv", index_col = 0)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 1598
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  type                  6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 710.6 KB


In [4]:
data["type"].value_counts()

0    4898
1    1599
Name: type, dtype: int64

## Séparation apprentissage / test

On sépare les données en apprentissage / test

In [5]:
y = data["type"]
x = data.drop("type", axis = 1)
x_train, x_test, y_train, y_test  = train_test_split(x, 
                                                     y, 
                                                     test_size=0.25, 
                                                     random_state=42)

## Apprentissage 

On crée l'objet à partir de la classe du modèle

In [9]:
modele_rf = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='sqrt',    
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
    ccp_alpha=0.0,
    max_samples=None,)

On ajuste le modèle sur les données d'apprentissage 

In [10]:
modele_rf.fit(x_train, y_train)


RandomForestClassifier()

On affiche les importances des variables :

In [11]:
pd.DataFrame(modele_rf.feature_importances_,
             index = x_train.columns, 
             columns = ["importance"]).sort_values(
    "importance", 
    ascending = False)

,importance
total sulfur dioxide,0.322018
chlorides,0.256535
volatile acidity,0.117677
density,0.064111
sulphates,0.052127
residual sugar,0.049076
free sulfur dioxide,0.048051
fixed acidity,0.040584
pH,0.021258
citric acid,0.017611


## Validation du modèle

On valide le modèle sur les données de test :

In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [13]:
print(f"Le pourcentage de bien classés est de : {accuracy_score(y_test, modele_rf.predict(x_test))*100} %")

Le pourcentage de bien classés est de : 99.63076923076923 %


On a un modèle extrêment efficace. On peut ragrder la matrice de confusion :

In [14]:
pd.DataFrame(confusion_matrix(y_test, modele_rf.predict(x_test)),
            index = ["blanc_données", "rouge_données"],
            columns = ["blanc_predit", "rouge_predit"])

,blanc_predit,rouge_predit
blanc_données,1239,0
rouge_données,6,380


On peut maintenant utiliser ce modèle pour faire de la prédiction :

In [15]:
modele_rf.predict(x_test)

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

Pour aller plus loin, on pourra utiliser des outils comme `GridSearchCV` pour l'ajustement des hyperparamètres du modèle.